https://contest.yandex.ru/contest/12899/problems/

Файлы из описания можно взять тут — https://yadi.sk/d/uHe-XpI7pcnZKA

План:
1. Разбираемся с implicit.als, играем настройками. https://medium.com/wttj-tech/exploring-collaborative-filtering-for-job-recommendations-91f09b7b536c
2. Пробуем дополнить матрицу лайками и избранным.
3. Пробуем из предсказания удалить те картинки, которые уже были просмотрены. Это уже было сделано.

Попытки:
1. Базовое решение, 4.2 балла
2. Увеличиваем число итераций als.fit c 30 до 35, 4.29. Смысла бороться за сотые нет, там лидеры к 30 баллам приближаются.
3. Умножаем матрицу на коэффициент alpha, как это делают авторы статьи и метода http://yifanhu.net/PUB/cf.pdf. Первая проба 15 (получил сразу 7.14), вторая 40 (6.34). Вернул до 30 итераций, иначе ноут выключается.
4. alpha = 10, iters = 30. 6.62.
5. aplha = 15, iters = 30.
6. aplha = 15, factors=10, iters = 35.
7. aplha = 15, factors=20, regularization=0.1, iterations=20. Добавил явно filter_already_liked_items=True, хотя параметр по умолчанию должен стоять. Это для того, чтобы в предсказаниях не было картинок, которые пользователь уже выбрал ранее. Этих рекомендаций по условию задачи не должно быть. 6.89 (посылка 20650335)
8. aplha = 15, factors=20, regularization=0.1, iterations=35. Сбой.
9. Надоело запускать ноут, нужно железо правильно готовить. И не могу повторить опыт 7.14, на будущее - нужно все версии исходников коммитить.
10. alpha = 15, factors=10, iters = 30. Добавляю train_likes.zip с весом 1 и train_shares.zip с весом 1. Просто добавляю строки при формировании coo_matrix. Получил 5.49 (посылка 20650558). Плохо, значит alpha вредит в этом случае.
11. aplha = 5, factors=10, iters = 30. Добавляю train_likes.zip с весом 1 и train_shares.zip с весом 1. 7.36 (посылка 20650720).
12. Избранное - важнее. Пробуем вес share = 2. aplha = 5, factors=10, iters = 30. Добавляю train_likes.zip с весом 1 и train_shares.zip с весом 2. Сбой.
13. **aplha = 5, factors=20, regularization=0.1, iterations=20. Добавляю train_likes.zip с весом 1 и train_shares.zip с весом 2. 11.33 (посылка 20651056).**
14. alpha = 3. factors=20, regularization=0.1, iterations=20. Добавляю train_likes.zip с весом 2 и train_shares.zip с весом 3. 10.35 (посылка 20651210).
15. aplha = 5, factors=20, regularization=0.01, iterations=20. Добавляю train_likes.zip с весом 1 и train_shares.zip с весом 2. 11.03. (посылка 20651388).
16. aplha = 3, factors=20, regularization=0.1, iterations=20. Добавляю train_likes.zip с весом 1 и train_shares.zip с весом 2. 11.04. (посылка 20651492).
17. aplha = 5, factors=20, regularization=0.1, iterations=25. Добавляю train_likes.zip с весом 1 и train_shares.zip с весом 2. 10.89. (посылка 20651580).
18. aplha = 5, factors=25, regularization=0.1, iterations=20. Добавляю train_likes.zip с весом 1 и train_shares.zip с весом 2. . (посылка 20651580).


Что не учитываем:
1. Дату. Пока не умею.

In [1]:
import pandas as pd
import numpy as np

from scipy.sparse import coo_matrix

import implicit

Читаем клики пользователей

In [2]:
train_clicks = pd.read_csv('train_clicks.csv')
print(train_clicks.count())
# train_clicks[:17]

user_id       2360862
picture_id    2360862
day           2360862
dtype: int64


Пробуем дополнить клики лайками и избранным

In [3]:
alpha = 5

train_likes = pd.read_csv('train_likes.csv')
train_shares = pd.read_csv('train_shares.csv')

train_clicks = pd.concat([train_clicks, train_likes])
# train_clicks = pd.concat([train_clicks, train_likes])

train_clicks = pd.concat([train_clicks, train_shares])
train_clicks = pd.concat([train_clicks, train_shares])
# train_clicks = pd.concat([train_clicks, train_shares])

In [4]:
users = train_clicks.user_id
items = train_clicks.picture_id

In [5]:
# По условиям задачи было 200000 пользователей, а реально Яндекс обманул - дал 445357
# users.value_counts()
users.nunique()

446188

Создаём разреженную матрицу пользователь — объект. Используем модель **coo_matrix**, она позволяет быстро сформировать матрицу, зная координаты значений

In [6]:
user_item = coo_matrix((np.ones_like(users), (users, items)))*alpha

Вывод разреженной матрицы

In [7]:
# Вот это жестко, когда третий элемент имеет col index = 1242875
# print(user_item.todense())
for row, col, value in zip(user_item.row, user_item.col, user_item.data):
    print("({0}, {1}) {2}".format(row, col, value))
    if row > 1000: break

(1442, 546149) 5


В качестве модели будем использовать разложение матрицы с помощью метода ALS

Но для начала убираем распараллеливание, иначе ноут выключается от таких нагрузок :)

In [8]:
# http://diracprogram.org/doc/release-12/installation/mkl.html
import os
print(os.environ.get('MKL_NUM_THREADS'))
if np.__config__.get_info('blas_mkl_info') and os.environ.get('MKL_NUM_THREADS') != '1':
    os.environ["MKL_NUM_THREADS"] = '1'
    os.environ["MKL_DYNAMIC"] = "FALSE"
    os.environ["OMP_DYNAMIC"] = "FALSE"
    os.environ["OMP_NUM_THREADS"] = "1"
    
print(os.environ.get('MKL_NUM_THREADS'))

None
1


In [9]:
# model = implicit.als.AlternatingLeastSquares(factors=10, iterations=30)
# model = implicit.als.AlternatingLeastSquares(factors=10, iterations=35)
model = implicit.als.AlternatingLeastSquares(factors=25, regularization=0.1, iterations=20)
# factors=20, regularization=0.1, iterations=20

In [10]:
model.fit(user_item.T.tocsr())

100%|██████████| 20.0/20 [01:41<00:00,  4.94s/it]


Прочитаем идентификаторы пользователей, для которых нам нужно сделать предсказания

In [11]:
test_users = pd.read_csv('test_users.csv')

In [12]:
user_item_csr = user_item.tocsr()

Для каждого пользователя найдём 100 самых релевантных изображений

In [13]:
rows = []
for user_id in test_users.user_id.values:
    items = [i[0] for i in model.recommend(user_id, user_item_csr, N=100, filter_already_liked_items=True)]
    rows.append(items)

Отформатируем идентификаторы как нужно для ответа

In [14]:
test_users['predictions'] = list(map(lambda x: ' '.join(map(str, x)), rows))

И запишем предсказания в файл

In [15]:
test_users.to_csv('predictions.csv', index=False)